<a href="https://colab.research.google.com/github/Nov05/DS-Unit-2-Tree-Ensembles/blob/master/module2-random-forests/kaggle_20190715.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-07-15
# Kaggle
# DS-Unit-2-Tree-Ensembles/
# module1-decision-trees/
# module2-random-forests/
competition_name = "ds5-predictive-modeling-challenge"

https://www.kaggle.com/c/ds5-predictive-modeling-challenge/leaderboard  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
# do this only once unless you create a new API key:
# create a folder with name "config" in "My Drive" 
# and store your API config file "kaggle.json" there

# do this every time after you reset all your runtimes,
# or kaggle won't find your API config file "kaggle.json" 
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/config

env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/config


In [0]:
# run this code block to download competition files
!kaggle competitions download -c ds5-predictive-modeling-challenge
# unzip all .zip files
!unzip \*.zip

# Imports

In [0]:
!pip install --upgrade pandas-profiling
!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import category_encoders as ce

import sklearn
from sklearn import datasets
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, \
     mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, \
     GridSearchCV
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, \
     OneHotEncoder, OrdinalEncoder, LabelEncoder

import category_encoders as ce
from category_encoders.binary import BinaryEncoder

%matplotlib inline

In [8]:
print("pandas_profiling", pandas_profiling.__version__)
print("category_encoders", ce.__version__)
print("pandas", pd.__version__)
print("sklearn", sklearn.__version__)

pandas_profiling 2.1.2
category_encoders 2.0.0
pandas 0.24.2
sklearn 0.21.2


# Download data

In [0]:
LOCAL = '../data/tanzania/'
# WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'
WEB = '' # if you have downloaded the files to Colab folder '/content'

train_features = pd.read_csv(WEB + 'train_features.csv')
train_labels = pd.read_csv(WEB + 'train_labels.csv')
test_features = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [10]:
X_train = train_features
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.80, test_size=0.20,
    stratify=y_train, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

# Modeling

In [51]:
numeric_features = X_train.select_dtypes(include=np.number).columns.to_list()
# categorical_features = X_train.select_dtypes(exclude=np.number).columns.to_list()
categorical_features, boolean_features = [], []
for col in X_train.select_dtypes(exclude=np.number):
  if isinstance(X_train[col][0], bool):
    boolean_features.append(col)
  else:
    categorical_features.append(col)
len(numeric_features), len(categorical_features), len(boolean_features)

(10, 28, 2)

In [52]:
# select high cardinality features
# for ordinal encoding (rather than one-hot encoding)
highcardi_features, lowcardi_features = [], []
for col in X_train[categorical_features]:
  if len(X_train[col].value_counts()) >= 10:
    highcardi_features.append(col)
  else:
    lowcardi_features.append(col)
len(numeric_features), len(boolean_features), len(highcardi_features), len(lowcardi_features)

(10, 2, 14, 14)

In [54]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=True, handle_unknown='ignore'))]) # sparse=False would exaust the memory
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', ce.OrdinalEncoder())])
boolean_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('boo', boolean_transformer, boolean_features),
        ('one', onehot_transformer, lowcardi_features), # categorical_features
        ('ord', ordinal_transformer, highcardi_features),
    ])

boost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
parameters = {'n_estimators': (1, 50),
              'base_estimator__max_depth': (1, 26),
              'algorithm': ('SAMME', 'SAMME.R')}

clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('pca', PCA(n_components=5, whiten=True)),
#                       ('svd', TruncatedSVD(n_components=200, random_state=5)), # 200059 cols
#                       ('lr', LogisticRegression(solver='lbfgs', multi_class='auto', n_jobs=-1)),
#                       ('dt', DecisionTreeClassifier(max_depth=26)),
#                       ('gscv', GridSearchCV(boost, parameters)) # might take long time
                      ('rf', RandomForestClassifier(n_estimators=100, random_state=5))
                     ])
clf.fit(X_train, y_train)

# print('TruncatedSVD explained variance ratio:', clf['svd'].explained_variance_ratio_.sum())
# print("Decision Tree feature importances:", clf['dt'].feature_importances_)
print("Validation score: %.3f" % clf.score(X_val, y_val))

Validation score: 0.811


rf, ordinal_transformer  
Validation score: 0.811   
<br>
('rf', RandomForestClassifier(n_estimators=100, random_state=5))  
Validation score: 0.810  
('rf', RandomForestClassifier())  
Validation score: 0.792  
('rf', RandomForestClassifier(n_estimators=100, max_depth=26, random_state=5))  
Validation score: 0.693  
<br>
('dt', DecisionTreeClassifier())   # depth=142  
Validation score: 0.780  
('dt', DecisionTreeClassifier(max_depth=75))  
Validation score: 0.770  
('dt', DecisionTreeClassifier(max_depth=15))  
Validation score: 0.715   
max_depth=29,30 / 0.782  
max_depth=28 / 0.784  
max_depth=27 / 0.786  
max_depth=26 / 0.782  
<br>
boost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  
('gscv', GridSearchCV(boost, parameters))  
alidation score: 0.694  
<br>
('svd', TruncatedSVD(n_components=2000, random_state=5)), # 200059  
TruncatedSVD explained variance ratio: 0.8524638115439054  
Validation score: 0.773


In [0]:
# print(len(clf['dt'].feature_importances_))
# print(clf['dt'].max_features_)
# print(clf['dt'].get_depth())
# print(clf['dt'].feature_importances_[:15])

# Submission

In [0]:
test_pred = clf.predict(test_features)
submission = sample_submission.copy()
submission['status_group'] = test_pred
submission_filename = 'submission.csv'
submission.to_csv(submission_filename, index=False)
print(submission.shape)

In [0]:
# !kaggle competitions submit ds5-predictive-modeling-challenge -f submission.csv -m "random forest all features n_estimators=100 81%"

Adopting a Hypothesis-Driven Modeling Workflow  
https://outline.com/5S5tsB  

# Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  

# AdaBoostClassifier

https://programtalk.com/python-examples/sklearn.tree.DecisionTreeRegressor/  
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html  
https://www.datacamp.com/community/tutorials/adaboost-classifier-python  

In [0]:
# iris = datasets.load_iris()
# boston = datasets.load_boston()

# def test_gridsearch():
#     # Check that base trees can be grid-searched.
#     # AdaBoost classification
#     boost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
#     parameters = {'n_estimators': (1, 2),
#                   'base_estimator__max_depth': (1, 2),
#                   'algorithm': ('SAMME', 'SAMME.R')}
#     clf = GridSearchCV(boost, parameters)
#     clf.fit(iris.data, iris.target)
 
#     # AdaBoost regression
#     boost = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
#                               random_state=0)
#     parameters = {'n_estimators': (1, 2),
#                   'base_estimator__max_depth': (1, 2)}
#     clf = GridSearchCV(boost, parameters)
#     clf.fit(boston.data, boston.target)

# test_gridsearch()

# TPOT

http://epistasislab.github.io/tpot/  